In [ ]:

import argparse
import numpy as np
import tensorflow as tf
import anndata
from tensorflow.keras.optimizers import Adam
from tqdm.keras import TqdmCallback

from CopyGATpreprocess import filter_and_normalize_data
from CopyGATbinning import build_gene_map, bin_genes_from_anndata
from CopyGAT import CopyGAT
from CopyGATclustering import find_normal_cluster, find_clones_gmm
from CopyGATsegmentation import generate_clone_profile
# from CopyGATsegmentation import generate_clone_profile_vrg

In [ ]:
#! /usr/bin/env python3

import argparse
import numpy as np
import anndata
from tensorflow.keras.optimizers import Adam
from tqdm.keras import TqdmCallback
from CopyGAT.CopyGATpreprocess import filter_and_normalize_data
from CopyGAT.CopyGATbinning import build_gene_map, bin_genes_from_anndata
from CopyGAT.CopyGAT import CopyGAT
from CopyGAT.CopyGATclustering import find_clones_kmeans, find_normal_cluster
from CopyGAT.CopyGATsegmentation import generate_clone_profile
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tqdm.keras import TqdmCallback
from sklearn.preprocessing import normalize
import scipy.sparse as sp
from sklearn.neighbors import NearestNeighbors
def construct_knn_graph(features, k=10):
    nn = NearestNeighbors(n_neighbors=k, metric='cosine').fit(features)
    adj_matrix = nn.kneighbors_graph(features)
    return adj_matrix

def pad_tensor(tensor, target_shape):
    if isinstance(tensor, tf.SparseTensor):
        tensor = tf.sparse.to_dense(tensor)
    padding_sizes = [(0, target_shape[i] - tensor.shape[i]) for i in range(len(target_shape))]
    return tf.pad(tensor, padding_sizes)
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

def train_neural_network(data, adj, max_cp, intermediate_dim, latent_dim, batch_size, epochs, lr=1e-3, eps=0.01):
    features = tf.convert_to_tensor(data, dtype=tf.float32)

    model = CopyGAT(data.shape[-1], intermediate_dim, latent_dim, max_cp)

    optimizer = tf.keras.optimizers.Adam(learning_rate=lr, epsilon=eps)
    model.compile(optimizer=optimizer)
    num_samples = features.shape[0]
    epoch_loss_history = []
    return model

In [ ]:
def compute_pseudo_copy(x_bin, norm_mask):
    confident_norm_x = x_bin[norm_mask]
    print(np.shape(confident_norm_x))
    baseline = np.median(confident_norm_x, axis=0)
    baseline[baseline == 0] = 1
    pseudo_cp = x_bin / baseline * 2
    pseudo_cp[norm_mask] = 2.
    pseudo_cp = tf.convert_to_tensor(pseudo_cp, dtype='float')
    return pseudo_cp



In [ ]:
def split_data_by_cluster(data, pred_label, ncell_index):

    unique_labels = np.unique(pred_label)

    unique_labels = unique_labels[unique_labels != ncell_index]

    split_data = {}

    for label in unique_labels:
        mask = (pred_label == label)
        split_data[label] = data[mask]

    return split_data

In [ ]:
def perform_segmentation(clone_cn, chrom_list, eta):


    segment_cn, breakpoints = generate_clone_profile(clone_cn, chrom_list, eta)
    cell_cn = clone_cn.numpy()
    st = 0
    for i in range(1, len(breakpoints)):
        ed = breakpoints[i]
        m = np.mean(cell_cn[:, st:ed], axis=1)
        cell_cn[:, st:ed] = np.repeat(
                                        np.reshape(m, (1, m.size)).T,
                                        ed - st,
                                        axis=1)
        st = ed

    m = np.mean(cell_cn[:, st:], axis=1)
    cell_cn[:, st:] = np.repeat(
                                    np.reshape(m, (1, m.size)).T,
                                    np.shape(cell_cn)[1] - st,
                                    axis=1)

    return segment_cn, breakpoints, cell_cn


In [ ]:
def normalize_adj(adj_tf):

    adj = sp.coo_matrix((adj_tf.values.numpy(), (adj_tf.indices.numpy()[:, 0], adj_tf.indices.numpy()[:, 1])),
                         shape=adj_tf.dense_shape.numpy())

    adj = adj + sp.eye(adj.shape[0])

    row_sum = np.array(adj.sum(1)).flatten()  
    d_inv_sqrt = np.power(row_sum, -0.5)
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0. 

    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)

    adj_normalized = d_mat_inv_sqrt @ adj @ d_mat_inv_sqrt

    adj_coo = sp.coo_matrix(adj_normalized)
    adj_tf_normalized = tf.sparse.SparseTensor(indices=np.array([adj_coo.row, adj_coo.col]).T,
                                               values=adj_coo.data,
                                               dense_shape=adj_coo.shape)
    adj_tf_normalized = tf.sparse.reorder(adj_tf_normalized)  

    return adj_tf_normalized




In [ ]:
import numpy as np
import tensorflow as tf
import scipy.sparse as sp
from sklearn.metrics import pairwise_distances
def run_pipeline(umi_counts, cell_cycle_gene_list, bin_size, max_cp, intermediate_dim, latent_dim, batch_size, epochs, number_of_clones):
    feature_names=['gene_ids','gene_symbol']
    adata = filter_and_normalize_data(umi_counts,
                                      cell_cycle_gene_list,
                                      alternative_feature_names=feature_names)
    server_url = 'http://www.ensembl.org'
    attributes = ['external_gene_name',
                  'ensembl_gene_id',
                  'chromosome_name',
                  'start_position',
                  'end_position']
    gene_map = build_gene_map(server_url, attributes)
    data, chrom_list = bin_genes_from_anndata(adata, bin_size, gene_map)
    with open('chrom_list.npy', 'wb') as f:
        np.save(f, chrom_list)
    gene_map = build_gene_map(server_url, attributes)
    np.save('gene_map.npy', gene_map)
    print('finished preprocess')

    # train cluster NNs
    try: 
        x = data.X.todense()
    except AttributeError:
        x = data.X
    x_bin = x.reshape(-1,bin_size).copy().mean(axis=1).reshape(x.shape[0],-1) 
    features = tf.convert_to_tensor(x_bin, dtype=tf.float32)
    adj = construct_knn_graph(features, k=10)
    adj = sp.csr_matrix(adj)
    adj = adj.tocoo()
    adj = tf.sparse.SparseTensor(indices=np.array([adj.row, adj.col]).T,
                                 values=adj.data,
                                 dense_shape=adj.shape)
    adj = tf.sparse.reorder(adj)
    adj = normalize_adj(adj)
    clus_model = train_neural_network(x_bin,
                                      adj,
                                      max_cp,
                                      intermediate_dim,
                                      latent_dim,
                                      batch_size,
                                      epochs)

    print('CopyGAT Idetifying normal cells...')
    m,v,z = clus_model.z_encoder([x_bin,adj])
    pred_label_gmm = find_clones_gmm(z, n_clones=number_of_clones)
    cluster_auto_corr_gmm, ncell_index_gmm = find_normal_cluster(x_bin, pred_label_gmm)
    normal_cells_gmm = np.sum(pred_label_gmm == ncell_index_gmm)
果
    pred_label = pred_label_gmm
    cluster_auto_corr = cluster_auto_corr_gmm
    ncell_index = ncell_index_gmm
    norm_mask = (pred_label == ncell_index)
    norm_x = compute_pseudo_copy(x_bin, norm_mask)
    print(' start Training CopyGAT')
    copygat = train_neural_network(norm_x,
                                   adj,
                                   max_cp,
                                   intermediate_dim,
                                   latent_dim,
                                   batch_size,
                                   epochs,
                                   eps=0.02)

    _, _, latent_z = copygat.z_encoder([norm_x,adj])
    copy_bin = copygat.encoder([norm_x,latent_z])

    data.obsm['latent'] = z
    data.obsm['copy_number'] = copy_bin
    with open('copy.npy', 'wb') as f:
        np.save(f, copy_bin)
    cluster_dict = split_data_by_cluster(copy_bin, pred_label, ncell_index)
    print('Segmantation')
    for label, cluster_data in cluster_dict.items():
        segment_cn, breakpoints, sc_cn = perform_segmentation(cluster_data, chrom_list, eta=6)
        filename = f"CopyGATclone_{label}_single_cell_profile.npy"
        with open(filename, 'wb') as f:
            np.save(f, sc_cn)
        final_prof = np.repeat(segment_cn, bin_size)
        filename = f"CopyGATclone_{label}_profile.npy"
        with open(filename, 'wb') as f:
            np.save(f, final_prof)
        filename = f"CopyGATclone_{label}_breakpoints.npy"
        with open(filename, 'wb') as f:
            np.save(f, breakpoints)

    return None

In [ ]:
def main(input,cell_cycle_genes):

    parser = argparse.ArgumentParser()
    parser.add_argument('input', help="input UMI")
    parser.add_argument('cell_cycle_genes', help="path of list of cell cycle genes")
    parser.add_argument('-g', '--gpu', type=int, help="GPU id")
    parser.add_argument('-bin', '--bin_size', type=int, help="bin size")
    parser.add_argument('-mc', '--max_cp', type=int, help="maximum copy number")
    parser.add_argument('-intd', '--intermediate_dim', type=int, help="intermediate dimension")
    parser.add_argument('-l', '--latent_dim', type=int, help="latent dim")
    parser.add_argument('-bs', '--batch_size', type=int, help="batch size")
    parser.add_argument('-ep', '--epochs', type=int, help="number of epochs")
    parser.add_argument('-nc', '--number_of_clones', type=int, help="number_of_clones")

    args = parser.parse_args()
    args.input=input
    file = args.input
    args.cell_cycle_genes=cell_cycle_genes
    cc_genes = args.cell_cycle_genes

    if args.gpu:
        dvc = '/device:GPU:{}'.format(args.gpu)
    else:
        dvc = '/device:GPU:0'

    if args.bin_size:
        bin_size = args.bin_size
    else:
        bin_size = 25

    if args.max_cp:
        max_cp = args.max_cp
    else:
        max_cp = 15

    if args.intermediate_dim:
        intermediate_dim = args.intermediate_dim
    else:
        intermediate_dim = 128

    if args.latent_dim:
        latent_dim = args.latent_dim
    else:
        latent_dim = 6

    if args.batch_size:
        batch_size = args.batch_size
    else:
        batch_size = 128

    if args.epochs:
        epochs = args.epochs
    else:
        epochs = 200

    if args.number_of_clones:
        number_of_clones = args.number_of_clones
    else:
        number_of_clones = 2

    with tf.device(dvc):
        a=run_pipeline(file, cc_genes, bin_size, max_cp, intermediate_dim, latent_dim, batch_size, epochs, number_of_clones)


In [ ]:

file="scRNA-seq.h5ad"
cc_genes="cell division cycle gene"
a=run_pipeline(file, cc_genes, bin_size=25, max_cp=15, intermediate_dim=128, latent_dim=20, batch_size=128, epochs=200, number_of_clones=2)